In [ ]:
import cv2

# Object tracking function using Mean Shift or CamShift
def track_obj(video_path, trtype):
    cap = cv2.VideoCapture(video_path)
    
    # Initialize tracker
    if trtype == 'MeanShift':
        # For Mean Shift, we use cv2.meanShift() function
        ret, frame = cap.read()
        bbox = cv2.selectROI("Tracking", frame, False)
        x, y, w, h = bbox
        track_window = (x, y, w, h)
        roi = frame[y:y+h, x:x+w]
        hsv_roi = cv2.cvtColor(roi, cv2.COLOR_BGR2HSV)
        mask = cv2.inRange(hsv_roi, (0., 60., 32.), (180., 255., 255.))
        roi_hist = cv2.calcHist([hsv_roi], [0], mask, [180], [0, 180])
        cv2.normalize(roi_hist, roi_hist, 0, 255, cv2.NORM_MINMAX)
        term_criteria = (cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 10, 1)
        while True:
            ret, frame = cap.read()
            if not ret:
                break
            hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
            dst = cv2.calcBackProject([hsv], [0], roi_hist, [0, 180], 1)
            ret, track_window = cv2.meanShift(dst, track_window, term_criteria)
            x, y, w, h = track_window
            cv2.rectangle(frame, (x, y), (x + w, y + h), (255,0, 0), 2)
            cv2.imshow("Tracking", frame)
            if cv2.waitKey(1) & 0xFF == ord('q'):
                break
    elif trtype == 'CamShift':
        # For CamShift, we use cv2.CamShift() function which is an extension of Mean Shift
        ret, frame = cap.read()
        bbox = cv2.selectROI("Tracking", frame, False)
        x, y, w, h = bbox
        track_window = (x, y, w, h)
        roi = frame[y:y+h, x:x+w]
        hsv_roi = cv2.cvtColor(roi, cv2.COLOR_BGR2HSV)
        mask = cv2.inRange(hsv_roi, (0., 60., 32.), (180., 255., 255.))
        roi_hist = cv2.calcHist([hsv_roi], [0], mask, [180], [0, 180])
        cv2.normalize(roi_hist, roi_hist, 0, 255, cv2.NORM_MINMAX)
        term_criteria = (cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 10, 1)
        while True:
            ret, frame = cap.read()
            if not ret:
                break
            hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
            dst = cv2.calcBackProject([hsv], [0], roi_hist, [0, 180], 1)
            ret, track_box = cv2.CamShift(dst, track_window, term_criteria)
            pts = cv2.boxPoints(ret)
            pts = np.int0(pts)
            cv2.polylines(frame, [pts], True, (0,0, 255), 2)
            cv2.imshow("Tracking", frame)
            if cv2.waitKey(1) & 0xFF == ord('q'):
                break
    else:
        print("Tracker type not supported.")
        return
    
    cap.release()
    cv2.destroyAllWindows()


# Experiment 1: Moving camera with static object
track_obj(r"box.mp4",'MeanShift')

# Experiment 2: Moving camera with moving object
track_obj(r"C:/Desktop/SP24/CV/final projects/object tracking/Moving_camera_with_moving_obj.mp4", 'MeanShift')

# Experiment 3: Static camera with moving object
track_obj(r"C:/Desktop/SP24/CV/final projects/object tracking/static_camera_with_moving_obj.mp4", 'MeanShift')
